In [15]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

#Import Data
dataset=pd.read_csv('train.csv',sep=',',header=0)

In [16]:
dataset['condition_numeric'] = dataset.condition.map({
   'Fair': 0,
   'Good': 1,
   'Like New': 2
})
dataset['area_number_numberic'] = dataset.area_name.map({
   'fff': 0,
   'aaa': 1,
   'ddd': 2,
   'jjj': 3,
   'hhh': 4,
   'bbb': 5,
   'ccc': 6,
   'ggg': 7,
   'kkk': 8,
   'eee': 9
})
dataset['day_number_numberic'] = dataset.day.map({
   'Sun': 0,
   'Mon': 1,
   'Tue': 2,
   'Wed': 3,
   'Thu': 4,
   'Fri': 5,
   'Sat': 6
})


In [18]:
X = dataset[[
   'item_id', 'sold_price', 'price', 'area_number_numberic',
   'condition_numeric', 'size' ,'day_number_numberic', 'hour'
]]
y = dataset.category_class
y = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state=20)

In [19]:
linear_classifier=LogisticRegression()
linear_classifier.fit(X_train,y_train)
LC_Predicted_Results=linear_classifier.predict(X_test)
print(metrics.accuracy_score(LC_Predicted_Results, y_test))

0.5285714285714286


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [20]:
random_classifier = RandomForestClassifier()
random_classifier.fit(X_train, y_train)
predicted_results = random_classifier.predict(X_test)
print(metrics.accuracy_score(predicted_results, y_test))

0.5214285714285715


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [21]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [22]:
 rf = RandomForestClassifier()
 rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
 rf_random.fit(X_train,y_train)
 print("tuned hpyerparameters :(best parameters) ",rf_random.best_params_)
 print("accuracy :",rf_random.best_score_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.9min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


tuned hpyerparameters :(best parameters)  {'n_estimators': 1400, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': True}
accuracy : 0.5803571428571429


In [23]:
random_classifier = RandomForestClassifier(n_estimators = 1400, min_samples_split = 5, min_samples_leaf = 4, max_features = 'sqrt', max_depth = 80, bootstrap= True)
random_classifier.fit(X_train, y_train)
predicted_results = random_classifier.predict(X_test)
print(metrics.accuracy_score(predicted_results, y_test))

0.5857142857142857


In [10]:
#Import Data
testset=pd.read_csv('test.csv',sep=',',header=0)

In [24]:
testset['condition_numeric'] = testset.condition.map({
   'Fair': 0,
   'Good': 1,
   'Like New': 2
})
testset['area_number_numberic'] = testset.area_name.map({
   'fff': 0,
   'aaa': 1,
   'ddd': 2,
   'jjj': 3,
   'hhh': 4,
   'bbb': 5,
   'ccc': 6,
   'ggg': 7,
   'kkk': 8,
   'eee': 9
})
testset['day_number_numberic'] = testset.day.map({
   'Sun': 0,
   'Mon': 1,
   'Tue': 2,
   'Wed': 3,
   'Thu': 4,
   'Fri': 5,
   'Sat': 6
})

In [25]:
X1 = testset[[
   'item_id', 'sold_price', 'price', 'area_number_numberic',
   'condition_numeric', 'size', 'day_number_numberic', 'hour'
]]

In [26]:
output = random_classifier.predict(X1)

In [27]:
print(output)

[2 0 1 1 2 0 0 0 0 1 2 0 0 3 0 2 3 1 0 2 2 0 2 0 1 0 4 2 2 0 0 1 2 0 1 0 2
 1 0 0 0 4 0 2 1 1 1 0 1 0 2 1 1 3 2 0 0 1 0 0 2 2 0 1 1 0 1 0 0 0 2 0 3 1
 0 0 1 2 2 0 0 4 1 0 0 4 2 1 1 1 3 2 1 2 3 2 0 0 0 0 3 0 3 2 0 2 4 1 3 1 0
 0 1 2 2 0 1 0 0 2 0 0 2 0 0 0 0 2 0 0 0 4 0 3 4 2 0 3 0 0 2 0 3 0 0 2 3 1
 0 0 0 0 0 3 2 3 3 1 0 0 0 3 2 4 0 2 2 1 2 4 0 0 0 2 1 1 2 0 1 1 2 0 3 2 1
 0 1 0 2 0 3 1 1 0 1 2 0 2 4 2 2 2 0 0 2 0 0 3 2 2 0 0 0 2 0 1 2 2 4 0 1 3
 2 0 0 2 1 0 2 1 0 0 1 0 1 4 1 1 2 0 4 0 0 0 2 1 3 0 2 3 1 0 0 1 1 2 4 0 3
 0 3 1 0 2 0 0 2 0 3 1 0 2 0 0 2 0 1 0 0 1 2 1 1 2 0 2 0 0 0 3 0 0 3 0 1 0
 3 1 2 1]
